# Fluxonium with multi-path coupling(Simultaneous X gate)

This example is based on Nguyen, L. B. et al. Blueprint for a High-Performance Fluxonium Quantum Processor. PRX Quantum 3, 037001 (2022).
We simulate a 6 fluxonium qubit system from an underlying periodic lattice.
Idling hamiltonian of the system is
$$ H(0) = \sum_i H_{\mathrm{f},i}  + \sum_{\langle i,j \rangle } H_{ij}. $$
Hamiltonian of single fluxonium is
$$ H_{\mathrm{f},i} = 4E_{\mathrm{C},i} n_i^2 + \frac{1}{2} E_{\mathrm{L},i} (\varphi_i +\varphi_{\mathrm{ext},i})^2
    -E_{\mathrm{J},i}\cos \left( \varphi_i \right).$$
The coupling terms have the form
$$H_{ij} = J_{\mathrm{C}} n_i n_j - J_{\mathrm{L}} \varphi_i \varphi_j.$$
The couplings are chosen in a way such that the idling $ZZ$-crosstalk is almost zero.

In [1]:
import jax
import jax.numpy as jnp

import supergrad
from supergrad.helper import Evolve
from supergrad.utils import compute_fidelity_with_1q_rotation_axis
from supergrad.utils.gates import sigmax
from supergrad.utils.optimize import scipy_minimize, adam_opt
from supergrad.utils.format_conv import tree_print
from supergrad.utils.fidelity import pauli_diagnose_partition

from supergrad.scgraph.graph_mpc_fluxonium_5x5_periodic import XGatePeriodicGraphOpt

truncated_dim = 3  # how many levels we keep after diagonalization
add_random = True  # whether to add deviations to fluxonium parameters ec,ej,el
share_params = True  # whether we treat marked parameters as shared when computing gradients
unify_coupling = True  # set all couplings to be equal
compensation_option = 'only_vz'  # Use virtual-z compensation in fidelity computation

# Note all options above should be applied to the parameters before performing any real calculations

# instance the quantum processor graph, and choose a subset for time evolution
gate_graph = XGatePeriodicGraphOpt(1)

# Apply truncated_dim to fluxonium
gate_graph.set_all_node_attr(truncated_dim=truncated_dim)

# Apply random deviations to fluxonium parameters
if add_random:
    gate_graph.add_lcj_params_variance_to_graph()
qubit_subset = gate_graph.subscgraph(['q02', 'q03', 'q12', 'q13', 'q22', 'q23'])

# Set share_params and unify_coupling
qubit_subset.share_params = share_params
qubit_subset.unify_coupling = unify_coupling

# Construct the Evolve object for time evolution
evo = Evolve(qubit_subset)
target_unitary = supergrad.tensor(*([sigmax()] * len(qubit_subset.nodes)))


Let us print the parameters stored in the evo instance. These parameters are parsed from gate_graph. For how we construct this graph, please check `examples/fluxonium_multipath_coupling/graph_5x5_periodic.py` and the instructions inside.

In [2]:
# This is the same as  evo.all_params
params = qubit_subset.convert_graph_to_parameters()
tree_print(params)


{ 'edges': { 'q02:q03': { 'capacitive_coupling': {'strength': 0.07225663},
                          'inductive_coupling': {'strength': -0.01256637}}},
  'nodes': { 'q02': { 'compensation': { 'post_comp': -0.41941967,
                                        'pre_comp': 0.42591073},
                      'ec': 6.3532393823,
                      'ej': 25.1103087675,
                      'el': 6.344787656,
                      'pulse': { 'p1': { 'amp': 0.07467672,
                                         'length': 39.82459069,
                                         'omega_d': 3.70771137,
                                         'phase': 0.7558433}}},
             'q03': { 'compensation': { 'post_comp': -0.11531302,
                                        'pre_comp': 0.11757419},
                      'ec': 6.2550610547000005,
                      'ej': 25.1411401038,
                      'el': 5.000964059,
                      'pulse': { 'p1': { 'amp': 0.06839676,
                

In [3]:
def comp_u(params, target_unitary):
    # evolve system on the eigen basis.
    sim_u = evo.eigen_basis(params)
    fidelity_vz, compensated_u = compute_fidelity_with_1q_rotation_axis(target_unitary,
                                                            sim_u,
                                                            opt_method=None)
    return compensated_u

compensated_u = comp_u(params, target_unitary)


In the `comp_u` function in the above cell, we begin by computing the result of the time evolution unitary, `sim_u`, for the given parameters in `params`. The computation involves determining the time-evolution unitary matrix in the eigenbasis of the idling Hamiltonian. Subsequently, we calculate a suitable virtual-Z compensation, to ensure that the resulting `compensated_u` closely resembles the matrix of simultaneous X gates.

We want to assess the level of correlation in the error. Let us define $U_\mathrm{error} = U_{\mathrm{compensated}}^{\dagger} U_{\mathrm{target}}$. By expanding $U_\mathrm{error}$ in the Pauli basis, we obtain $U_\mathrm{error} = \sum_{\vec{j}} a(\vec{j})P_{\vec{j}}$. In the following list, we will provide pairs of $(P_{\vec{j}}, |a(\vec{j})|^2)$. The values $|a(\vec{j})|^2$ can be interpreted as the probabilities of stochastic errors after performing Pauli twirling.
The colors of the text help visualize the weights of errors.

In [4]:
partition = [[0],[1],[2],[3],[4],[5]]
res = pauli_diagnose_partition(target_unitary, compensated_u, 20, partition, 'black')


IIIIII   9.9944e-01

IIIZIZ   1.2491e-04

IIIXIY   7.9678e-05

IIIYIY   3.5984e-05

ZIXIII   1.6916e-05

...

YIXXIZ   4.8337e-06

XZYIII   4.6187e-06

IZZZII   4.6052e-06

XIZYII   4.2224e-06

IYXZII   4.2039e-06

For optimization, let us use an object function based on the average gate fidelity with leakage. The formula is from Physical Review A 87, 022309 (2013).
$$
	C = 1 - \frac{1}{D(D + 1)}[Tr(PU^{\dagger}_{\text{sim}}P U_{\text{sim}}P)+|Tr(PU^{\dagger}_{\text{sim}} P U_{\text{target}})|^2],   
$$
where $P$ is the projector into the computational subspace $S$, and $D=2^n$ is the dimension of $S$.

In [5]:
def infidelity(params, target_unitary):
    # evolve system on the eigen basis.
    sim_u = evo.eigen_basis(params)
    # calculate fidelity
    fidelity_vz, _ = compute_fidelity_with_1q_rotation_axis(target_unitary,
                                                            sim_u,
                                                            opt_method=None)

    return jnp.log10(1 - fidelity_vz)


In [6]:
vg = jax.value_and_grad(infidelity)(params, target_unitary)
print('The infidelity of simultaneous X gates is ', 10**vg[0])


The infidelity of simultaneous X gates is  0.0005508189711008757


In [7]:
print('The gradient of log10(infidelity):')
tree_print(vg[1])


The gradient of log10(infidelity):
{ 'edges': { 'q02:q03': { 'capacitive_coupling': { 'strength': -0.5254040939255397},
                          'inductive_coupling': { 'strength': -95.3640200890742}}},
  'nodes': { 'q02': { 'compensation': { 'post_comp': 0.002912061010306388,
                                        'pre_comp': -0.002912061010533762},
                      'ec': 3.6308840077831386,
                      'ej': -1.1561727006668847,
                      'el': 3.1334022251140285,
                      'pulse': { 'p1': { 'amp': 0.09382427071570856,
                                         'length': -0.011382726347448776,
                                         'omega_d': 0.7530296645143271,
                                         'phase': 0.05968167697414312}}},
             'q03': { 'compensation': { 'post_comp': 0.025252227306026498,
                                        'pre_comp': -0.02525222730588439},
                      'ec': 0.9169665253218147,
             

Show the optimization procedure of control parameters

In [8]:
adam_opt(infidelity,
         params,
         args=(target_unitary,),
         options={
             'adam_lr': 1e-6,
             'steps': 3
         })


{'adam_lr': 1e-06, 'adam_lr_decay_rate': 1000, 'steps': 3, 'adam_b1': 0.9, 'adam_b2': 0.999}
step: 0
parameters:
{ 'edges': { 'q02:q03': { 'capacitive_coupling': {'strength': 0.07225663},
                          'inductive_coupling': {'strength': -0.01256637}}},
  'nodes': { 'q02': { 'compensation': { 'post_comp': -0.41941967,
                                        'pre_comp': 0.42591073},
                      'ec': 6.3532393823,
                      'ej': 25.1103087675,
                      'el': 6.344787656,
                      'pulse': { 'p1': { 'amp': 0.07467672,
                                         'length': 39.82459069,
                                         'omega_d': 3.70771137,
                                         'phase': 0.7558433}}},
             'q03': { 'compensation': { 'post_comp': -0.11531302,
                                        'pre_comp': 0.11757419},
                      'ec': 6.2550610547000005,
                      'ej': 25.1411401038,
    

{'edges': {('q02',
   'q03'): {'capacitive_coupling': {'strength': Array(0.07225963, dtype=float64)}, 'inductive_coupling': {'strength': Array(-0.01256337, dtype=float64)}}},
 'nodes': {'q02': {'compensation': {'post_comp': Array(-0.41941917, dtype=float64),
    'pre_comp': Array(0.42591023, dtype=float64)},
   'ec': Array(6.35323639, dtype=float64),
   'ej': Array(25.11031175, dtype=float64),
   'el': Array(6.34478467, dtype=float64),
   'pulse': {'p1': {'amp': Array(0.07467687, dtype=float64),
     'length': Array(39.82459369, dtype=float64),
     'omega_d': Array(3.70770889, dtype=float64),
     'phase': Array(0.75584081, dtype=float64)}}},
  'q03': {'compensation': {'post_comp': Array(-0.11531404, dtype=float64),
    'pre_comp': Array(0.11757521, dtype=float64)},
   'ec': Array(6.25505806, dtype=float64),
   'ej': Array(25.1411431, dtype=float64),
   'el': Array(5.00096106, dtype=float64),
   'pulse': {'p1': {'amp': Array(0.06839683, dtype=float64),
     'length': Array(39.8130292,